**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Outside we must switch on ingress

`minikube addons enable ingress`

In [1]:
kubectl get pods -A | grep --color=always -E '$|ingress'

NAMESPACE              NAME                                         READY   STATUS      RESTARTS      AGE
default                kube-bench--1-cqzmb                          0/1     Completed   0             92m
default                nginx-strict-5cdc48cf55-jmjhj                1/1     Running     0             18h
default                nginx-strict-5cdc48cf55-rr4kc                1/1     Running     0             18h
default                nginx-strict-5cdc48cf55-vz7ds                1/1     Running     0             18h
ingress-nginx          ingress-nginx-admission-create--1-spxt9      0/1     Completed   0             49s
ingress-nginx          ingress-nginx-admission-patch--1-xc9zd       0/1     Completed   1             49s
ingress-nginx          ingress-nginx-controller-69bdbc4d57-zmr7p    1/1     Running     0             49s
kube-system            coredns-bcc9dbffc-nxcjt                      1/1     Running     0             21h
kube-system            etcd-minikube          

In [3]:
kubectl delete deployment nginx-strict
kubectl delete svc nginx-strict


deployment.apps "nginx-strict" deleted
service "nginx-strict" deleted


In [4]:
kubectl get svc -A

NAMESPACE              NAME                                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)                      AGE
default                kubernetes                           ClusterIP   10.96.0.1       <none>        443/TCP                      24h
ingress-nginx          ingress-nginx-controller             NodePort    10.105.83.255   <none>        80:31216/TCP,443:30274/TCP   3m28s
ingress-nginx          ingress-nginx-controller-admission   ClusterIP   10.98.125.104   <none>        443/TCP                      3m28s
kube-system            kube-dns                             ClusterIP   10.96.0.10      <none>        53/UDP,53/TCP,9153/TCP       24h
kubernetes-dashboard   dashboard-metrics-scraper            ClusterIP   10.107.147.22   <none>        8000/TCP                     21h
kubernetes-dashboard   kubernetes-dashboard                 ClusterIP   10.107.251.36   <none>        80/TCP                       21h


In [5]:
kubectl create -f - <<EOF
apiVersion: apps/v1 
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
        ports:
        - containerPort: 80
EOF

deployment.apps/nginx-deployment created


In [6]:
kubectl get all

NAME                                    READY   STATUS              RESTARTS   AGE
pod/kube-bench--1-cqzmb                 0/1     Completed           0          95m
pod/nginx-deployment-7848d4b86f-chjjb   0/1     ContainerCreating   0          3s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   24h

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment   0/1     1            0           3s

NAME                                          DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-7848d4b86f   1         1         0       3s

NAME                   COMPLETIONS   DURATION   AGE
job.batch/kube-bench   1/1           8s         95m


In [7]:
kubectl create -f - <<EOF
kind: Service
apiVersion: v1
metadata:
  name: "nginx"
spec:
  selector:
    app: "nginx"
  ports:
    - protocol: "TCP"
      port: 80
      targetPort: 80
  type: LoadBalancer
EOF

service/nginx created


In [8]:
kubectl get svc

NAME         TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)        AGE
kubernetes   ClusterIP      10.96.0.1      <none>         443/TCP        24h
nginx        LoadBalancer   10.110.37.38   10.110.37.38   80:32314/TCP   41s


In [9]:
kubectl describe service nginx

Name:                     nginx
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=nginx
Type:                     LoadBalancer
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.110.37.38
IPs:                      10.110.37.38
LoadBalancer Ingress:     10.110.37.38
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  32314/TCP
Endpoints:                172.17.0.2:80
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [10]:
kubectl get pods --all-namespaces

NAMESPACE              NAME                                         READY   STATUS      RESTARTS      AGE
default                kube-bench--1-cqzmb                          0/1     Completed   0             96m
default                nginx-deployment-7848d4b86f-chjjb            1/1     Running     0             112s
ingress-nginx          ingress-nginx-admission-create--1-spxt9      0/1     Completed   0             5m34s
ingress-nginx          ingress-nginx-admission-patch--1-xc9zd       0/1     Completed   1             5m34s
ingress-nginx          ingress-nginx-controller-69bdbc4d57-zmr7p    1/1     Running     0             5m34s
kube-system            coredns-bcc9dbffc-nxcjt                      1/1     Running     0             21h
kube-system            etcd-minikube                                1/1     Running     0             25h
kube-system            kube-apiserver-minikube                      1/1     Running     0             25h
kube-system            kube-controller-

In [11]:
kubectl get svc --all-namespaces

NAMESPACE              NAME                                 TYPE           CLUSTER-IP      EXTERNAL-IP    PORT(S)                      AGE
default                kubernetes                           ClusterIP      10.96.0.1       <none>         443/TCP                      25h
default                nginx                                LoadBalancer   10.110.37.38    10.110.37.38   80:32314/TCP                 108s
ingress-nginx          ingress-nginx-controller             NodePort       10.105.83.255   <none>         80:31216/TCP,443:30274/TCP   5m43s
ingress-nginx          ingress-nginx-controller-admission   ClusterIP      10.98.125.104   <none>         443/TCP                      5m43s
kube-system            kube-dns                             ClusterIP      10.96.0.10      <none>         53/UDP,53/TCP,9153/TCP       25h
kubernetes-dashboard   dashboard-metrics-scraper            ClusterIP      10.107.147.22   <none>         8000/TCP                     21h
kubernetes-dashboard  

In [13]:
kubectl get configmap -n ingress-nginx

NAME                        DATA   AGE
ingress-controller-leader   0      5m53s
ingress-nginx-controller    1      6m11s
kube-root-ca.crt            1      6m11s
tcp-services                0      6m11s
udp-services                0      6m11s


In [15]:
kubectl get configmap -n  ingress-nginx  ingress-controller-leader -o yaml

apiVersion: v1
kind: ConfigMap
metadata:
  annotations:
    control-plane.alpha.kubernetes.io/leader: '{"holderIdentity":"ingress-nginx-controller-69bdbc4d57-zmr7p","leaseDurationSeconds":30,"acquireTime":"2022-01-19T10:20:55Z","renewTime":"2022-01-19T10:27:11Z","leaderTransitions":0}'
  creationTimestamp: "2022-01-19T10:20:55Z"
  name: ingress-controller-leader
  namespace: ingress-nginx
  resourceVersion: "64397"
  uid: 52e6007c-d9c3-4d07-9d2f-cfbd7f9831a9


In [16]:
kubectl get configmap -n  ingress-nginx ingress-nginx-controller -o yaml

apiVersion: v1
data:
  hsts: "false"
kind: ConfigMap
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","data":{"hsts":"false"},"kind":"ConfigMap","metadata":{"annotations":{},"labels":{"app.kubernetes.io/component":"controller","app.kubernetes.io/instance":"ingress-nginx","app.kubernetes.io/name":"ingress-nginx"},"name":"ingress-nginx-controller","namespace":"ingress-nginx"}}
  creationTimestamp: "2022-01-19T10:20:37Z"
  labels:
    app.kubernetes.io/component: controller
    app.kubernetes.io/instance: ingress-nginx
    app.kubernetes.io/name: ingress-nginx
  name: ingress-nginx-controller
  namespace: ingress-nginx
  resourceVersion: "63908"
  uid: b167c150-8204-44f1-a3cd-ae9ee287e3fd
